## Shape Files

Shapefile es un formato de almacenamiento de datos vectoriales de **Esri** (Environmental Systems Research Institute) para almacenar la ubicación, la forma y los atributos de las entidades geográficas. Un shapefile se almacena en un conjunto de archivos relacionados y contiene una clase de entidad.
A continuación cargaremos un archivo de extensón *.shp*.

**Cargamos el shape**

In [ ]:
import geopandas as gpd

In [ ]:
#Cargamos el shape
shp = gpd.read_file("../archivos_st_py/vector/PNLCampana.shp")

In [ ]:
#Vemos la tabla de atributos del shape
shp

**Visualizamos el Shape**

In [ ]:
#Graficamos el shape

shp.plot()

In [ ]:
#Visualizar de manera interactiva
shp.explore()

**Cambiar el CRS (Sistema de Referencia de Coordenadas) de un Shapefile**

Un **Coordinate reference system** CRS define cómo el mapa proyectado bidimensional es un SIG se relaciona con lugares reales en la tierra. 

In [ ]:
#Ver el CRS actual
shp.crs

Lo que queremos ahora es cambiar su **CRS** a **UTM** zona 19s utilizando su código EPSG. Es decir, lo tenemos en coordenadas geograficas, y lo queremos pasar a coordenadas UTM.

In [ ]:
shp_rep = shp.to_crs(epsg=32719)
shp_rep.crs

En la forma de graficar no cambia en nada, solo cambia el sistema de referencias.

In [ ]:
shp_rep.explore()

**Exportar Shapefile**

In [ ]:
shp_rep.to_file("../practica_jupyter/shape/la_campana_utm.shp")

**Limpiar Shape**

Puede ocurrir que un Shapefile tenga muchas columnas con datos que muchas veces son innecesarios, para esto podemos quedarnos tan solo con las columnas de información que necesitemos.

In [ ]:
#importamos Geopandas
import geopandas as gpd

In [ ]:
#Cargamos el shape
shp = gpd.read_file("../archivos_st_py/vector/snaspe.shp") 
shp

Vamos a quedarnos solo con las columnas que nos interesan, y ademas vamos a cambiarle sus nombres.

In [ ]:
# Seleccionamos las columnas
shp_li = shp[["st_area_sh","Nombre","Tipo_Snasp","Region","Cod_Region","geometry"]]
shp_li

In [ ]:
#Cambiar nombre a las columnas
shp_li2 = shp_li.rename(columns={"st_area_sh" : "area",
                                "Nombre" : "nombre",
                                "Tipo_Snasp" : "tipo",
                                "Region" : "region",
                                "Cod_Region" : "cod"})
shp_li2

In [ ]:
#Exportamos el shapefile limpio

shp_li2.to_file("../practica_jupyter/shape/pa_chile.shp")

## Analizar Shapefile

Para analizar un shapefile debemos utilizar la función *nombre_shapeile.describe()*, esto nos dará una descripción de las columnas numéricas, dandonos su promedio, mínimo, desviación estandar, etc.

In [ ]:
#Importamos geopandas
import geopandas as gpd

In [ ]:
#Cargamos el shapefile
pa_cl = gpd.read_file("../practica_jupyter/shape/pa_chile.shp")
pa_cl

In [ ]:
#Obtenemos una descripción de las columnas numéricas
pa_cl.describe()

Sin embargo podemos ver que en el out de arriba hay un error, no podemos obtener una descripción numérica de esta manera de la columna **cod** debido a que esta es una columna categórica que solo se utiliza para poder distinguir los parques.

Para esto vamos a modificar esta columna, **pasandola de dato numérico** a **dato categórico** mediante el comando *nombre_shapefile . assing ( )*.

In [ ]:
#Modificamos columna "cod" y la ponemos como dato categorico

pa_cl2 = pa_cl.assign(cod = pa_cl["cod"].astype("category")) 
pa_cl2.describe()

Como vemos, ahora solo tenemos una descripción numérica correcta.

In [ ]:
#Como repaso, para observar la forma de el dataframe
pa_cl2.shape

In [ ]:
#Visualizar los primeros 5 elementos de la tabla 
pa_cl2.head()

In [ ]:
#Visualizar los últimos 5 elementos de la tabla
pa_cl2.tail()

In [ ]:
#Seleccionar las primeras diez filas

pa_cl2.iloc[:10]

Un ejemplo interesante es poder agrupar según alguna columna, en este caso según las regiones y en base a eso ver el promedio del área en cada región.

In [ ]:
pa_reg_ar = pa_cl2.groupby("region").agg({"area":"mean"})
pa_reg_ar

Para ordenar estos datos en un orden especifico, podemos utilizar la función *nombre_shapefile.sort_values("columna", condición)*. En este caso vamos a ordenar según el área de mayor a menor.

In [ ]:
#Ordenamos de mayor a menor
pa_reg_ar.sort_values("area", ascending = False)

Otro ejemplo es poder agrupar según el "tipo", realizaremos un promedio de todas las áreas y agruparemos de mayor a menor..

In [ ]:
pa_cl2.groupby("tipo").agg({"area":"mean"}).sort_values("area", ascending = False)

## Filtros

Para poder filtrar datos de nuestro shapefile debemos utilizar el método *nombre_shape.query ( "condición" )*.

In [ ]:
#importamos geopandas
import geopandas as gpd

In [ ]:
#Cargamos el shape limpio
pa_cl = gpd.read_file("../practica_jupyter/shape/pa_chile.shp")
pa_cl

In [ ]:
#Filtramos la condición de parques con un area menor a 10
pa2 = pa_cl.query("area<100")
pa2

In [ ]:
pa2.explore()

Hay que tener en cuenta que el argumento del metodo *query(" ")* siempre va entre doble comillas, es por esto que si quisieramos ingresar una condición que contemple un string, este deberá ir entre comillas simples.

In [ ]:
pa3 = pa_cl.query("tipo == 'Parque'")
pa3.head()

In [ ]:
pa3.explore()

Podemos filtrar con dos condiciones:

In [ ]:
#Filtrar con dos condiciones, una numérica y una categorica

pa4 = pa_cl.query("area < 10 & tipo == 'Parque'")

pa4

**CSV o Excel a Shapefile**

En un archivo .csv o .xlsx podríamos tener una lista de puntos con sus coordenadas, estos puntos geograficos podemos representarlos viasualmente a traves de un Shapefile.

A continuación cargamos un archivo .csv con una serie de puntos y definimos su ubicación en el mapa.

In [ ]:
import pandas as pd 
import geopandas as gpd

In [ ]:
pts_csv = pd.read_csv("../archivos_st_py/csv/ptos.csv")
pts_csv.head()
#Con excel es lo mismo, solo sería read_excel

Esta lista, ademas de las coordenadas geográficas, también podría tener una descripción o cualidades acerca de cada uno de estos puntos. En este caso nos dice la cobertura de cada punto.

Creo una nueva shapefile a partir del csv.

In [ ]:
ptos_geo = gpd.GeoDataFrame(pts_csv,
                            geometry = gpd.points_from_xy(
                            pts_csv["x"],pts_csv["y"])).set_crs("epsg:32719")
                            

In [ ]:
ptos_geo.explore()

Por úlitmo, despues de convertir este archivo .csv en .shp, querriamos exportarlo como shape.

In [ ]:
#Exportar puntos en formato shapefile

ptos_geo.to_file("../practica_jupyter/shape/puntos.shp")